#### Pipeline ETL para ingesta en base de datos relacional

In [1]:
# importamos las librerias necesarias
import pandas as pd
import json
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import gzip

In [3]:
"""
# Ruta al archivo Parquet
ruta_archivo = ".\metadata.parquet"

# Leer el archivo Parquet en un DataFrame de Pandas
df_metadata = pd.read_parquet(ruta_archivo)
"""



'\n# Ruta al archivo Parquet\nruta_archivo = ".\\metadata.parquet"\n\n# Leer el archivo Parquet en un DataFrame de Pandas\ndf_metadata = pd.read_parquet(ruta_archivo)\n'

In [5]:
path = r"C:\Users\sebas\OneDrive\Escritorio\Proyecto-Grupal-Google-yelp\Sprint 2 Ingenieria de Datos\pipeline ETL\google\metadata-sitios\1.json"

# Cargar el archivo JSON en un DataFrame de Pandas
df_metadata = pd.read_json(path, orient='records', lines=True)

In [7]:
df_metadata.head(2)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.35710,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.21529,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [8]:
# Eliminamos columnas irrelevantes
df_metadata.drop(columns=['address', 'state', 'relative_results', 'url','hours','description', 'price'], inplace=True)

In [9]:
from pandas import json_normalize 

# Desanidar los datos de la columna "MISC"
df_desanidado = json_normalize(df_metadata['MISC'])

In [10]:
# Vamos a trabajar con una copia de metadata para crear una posible tabla "miselaneos"
gmap_id = df_metadata.copy()

# Resetear el índice del DataFrame df_gmap_id
gmap_id.reset_index(drop=True, inplace=True)

# Combinar los DataFrames desanidados
miselaneos = pd.concat([gmap_id, df_desanidado], axis=1)

In [11]:
# Eliminamos columna MISC
df_metadata.drop(columns=['MISC'], inplace=True)

In [15]:
# DEBERIAMOS QUEDARNOS UNICAMENTE CON EL PRIMER VALOR DE LA COLUMNA CATEGORY
df_metadata['category'] = df_metadata['category'].str[0]


In [16]:
df_metadata['category']

0                         Pharmacy
1                 Textile exporter
2                Korean restaurant
3                     Fabric store
4                     Fabric store
                    ...           
274996          Real estate agency
274997            Fence contractor
274998     Health insurance agency
274999    Building materials store
275000            Auto repair shop
Name: category, Length: 275001, dtype: object

In [17]:
# Verificar si hay valores duplicados
df_metadata.duplicated().value_counts()

False    248428
True      26573
Name: count, dtype: int64

In [18]:
# Eliminamos duplicados
df_metadata.drop_duplicates(inplace=True)

In [19]:
# Verificar si hay valores nulos
df_metadata.isnull().sum()

name                 7
gmap_id              0
latitude             0
longitude            0
category          2025
avg_rating           0
num_of_reviews       0
dtype: int64

In [20]:
df_metadata.shape

(248428, 7)

In [21]:
# cambio el nombre de la columna gmap_id a business_id 
df_metadata.rename(columns={'gmap_id': 'business_id'}, inplace=True)

In [22]:
# Verificamos tamaño de dataset
df_metadata.shape

(248428, 7)

In [23]:
# Observamos resultados
df_metadata.head(5)

,name,business_id,latitude,longitude,category,avg_rating,num_of_reviews
0,Porter Pharmacy,0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,Pharmacy,4.9,16
1,City Textile,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.215290,Textile exporter,4.5,6
2,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,Korean restaurant,4.4,18
3,Nova Fabrics,0x80c2c89923b27a41:0x32041559418d447,34.023669,-118.232930,Fabric store,3.3,6
4,Nobel Textile Co,0x80c2c632f933b073:0xc31785961fe826a6,34.036694,-118.249421,Fabric store,4.3,7


In [25]:
# importamos df_florida para hacer el merge final
df_florida = pd.read_csv(r"C:\Users\sebas\OneDrive\Escritorio\Proyecto-Grupal-Google-yelp\Sprint 2 Ingenieria de Datos\pipeline ETL\google\output_data\florida_table.csv")

In [28]:
df_florida.head(2)

,user_id,rating,business_id,timestamp,sentiment_analysis
0,1.014719e+20,1,0x8893863ea87bd5dd:0x9383ebf973e74abb,2021-03-08 15:07:30,0
1,1.154772e+20,5,0x8893863ea87bd5dd:0x9383ebf973e74abb,2020-07-18 00:13:37,0


In [29]:
merged_df = pd.merge(df_florida, df_metadata, on='business_id', how='inner')

In [33]:
merged_df.head()

,user_id,rating,business_id,timestamp,sentiment_analysis,name,latitude,longitude,category,avg_rating,num_of_reviews
0,1.014719e+20,1,0x8893863ea87bd5dd:0x9383ebf973e74abb,2021-03-08 15:07:30,0,"Brian Shaheen, MD",30.159982,-85.752277,Family practice physician,4.2,18
1,1.154772e+20,5,0x8893863ea87bd5dd:0x9383ebf973e74abb,2020-07-18 00:13:37,0,"Brian Shaheen, MD",30.159982,-85.752277,Family practice physician,4.2,18
2,1.018050e+20,5,0x8893863ea87bd5dd:0x9383ebf973e74abb,2018-05-04 10:30:53,2,"Brian Shaheen, MD",30.159982,-85.752277,Family practice physician,4.2,18
3,1.063444e+20,1,0x8893863ea87bd5dd:0x9383ebf973e74abb,2016-07-07 16:10:19,0,"Brian Shaheen, MD",30.159982,-85.752277,Family practice physician,4.2,18
4,1.008751e+20,5,0x8893863ea87bd5dd:0x9383ebf973e74abb,2016-02-12 12:56:55,2,"Brian Shaheen, MD",30.159982,-85.752277,Family practice physician,4.2,18


In [34]:
# Exportamos el dataset
merged_df.to_csv('merged.csv', index=False)
merged_df.to_parquet('merged.parquet')